In [49]:
import os

os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"
os.environ["TF_GPU_THREAD_MODE"] = "gpu_private"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf


print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import pandas as pd
from pathlib import Path
import numpy as np
import tempfile


TensorFlow version: 2.6.0


In [58]:
file = Path.cwd() / "eth_histdata.csv"
df = pd.read_csv(file)
df.set_index("time", inplace=True)
df.head()


,low,high,open,close,volume,Day sin,Day cos,Year sin,Year cos
time,,,,,,,,,
2021-10-01 21:00:00,3292.56,3294.21,3292.59,3293.52,7.525196,-0.707107,7.071068e-01,-0.999951,0.009902
2021-10-01 20:00:00,3279.00,3318.74,3299.20,3292.78,10941.407660,-0.866025,5.000000e-01,-0.999958,0.009185
2021-10-01 19:00:00,3286.06,3320.00,3294.89,3299.01,10209.782648,-0.965926,2.588190e-01,-0.999964,0.008469
2021-10-01 18:00:00,3252.86,3300.00,3278.60,3295.35,11702.589629,-1.000000,-4.354258e-12,-0.999970,0.007752
2021-10-01 17:00:00,3264.05,3286.53,3278.38,3278.65,8259.882894,-0.965926,-2.588190e-01,-0.999975,0.007035


In [59]:
def make_ds(data):
    data = np.array(data, dtype=np.float32)
    ds = tf.keras.preprocessing.timeseries_dataset_from_array(
        data=data[:-24],
        targets=data[:24],
        sequence_length=24,
        sequence_stride=1,
        shuffle=True,
        batch_size=32,
    )

    # ds = ds.map(self.split_window)

    return ds


In [60]:
def train_input_fn():
    file = str(Path.cwd() / "eth_histdata.csv")
    data = tf.data.experimental.make_csv_dataset(
        file, batch_size=32, label_name="close"
    )
    batches = data.cache().repeat().prefetch(tf.data.AUTOTUNE)

    return batches


In [64]:
df.columns

Index(['low', 'high', 'open', 'close', 'volume', 'Day sin', 'Day cos',
       'Year sin', 'Year cos'],
      dtype='object')

In [70]:
model_dir = tempfile.mkdtemp()
high = tf.feature_column.numeric_column("high")
low = tf.feature_column.numeric_column("low")
open = tf.feature_column.numeric_column("open")
volume = tf.feature_column.numeric_column("volume")
cols = list(df.columns)
cols.remove("close")

for col in cols:
    col = tf.feature_column.numeric_column(col)

print(cols)


['low', 'high', 'open', 'volume', 'Day sin', 'Day cos', 'Year sin', 'Year cos']


In [62]:
model = tf.estimator.LinearClassifier(
    model_dir=model_dir, n_classes=2, feature_columns=cols
)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\jadav\\AppData\\Local\\Temp\\tmpo46fol2w', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [63]:
model = model.train(input_fn=train_input_fn, steps=100)

INFO:tensorflow:Calling model_fn.


ValueError: Received a feature column from TensorFlow v1, but this is a TensorFlow v2 Estimator. Please either use v2 feature columns (accessible via tf.feature_column.* in TF 2.x) with this Estimator, or switch to a v1 Estimator for use with v1 feature columns (accessible via tf.compat.v1.estimator.* and tf.compat.v1.feature_column.*, respectively.